<a href="https://colab.research.google.com/github/hereagain-Y/TCR_VAE/blob/main/Dataloader_of_bag_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install Sparsemax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import random
import itertools
import numpy as np
import itertools
import pandas as pd 
import random
import time
from datetime import datetime
from pkgutil import extend_path
import numpy as np
import matplotlib.pyplot as plt

from six.moves import xrange
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np

from tqdm import tqdm
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
#
from scipy.spatial import distance
import random 
from scipy import spatial
from scipy import stats
from sparsemax import Sparsemax
from torch.utils.data import TensorDataset, DataLoader
import os
import glob

In [19]:
cuda = False
device = torch.device("cuda" if cuda else "cpu")
n_feature = 16   


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class VAE(nn.Module):
    def __init__(self,h_dim=64*10*10, z_dim=32):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            
            nn.Conv2d(1, 16, kernel_size=5, stride=1), #16
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=5, stride=1), # 12
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1), #10
            nn.ReLU()
        
        )

        
        
        # mean 64*5*5 =
        self.fc1 = nn.Linear(h_dim, z_dim)
        # var 
        #self.fc2 = nn.Linear(h_dim, z_dim)
        # for decoder layer 
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.decoder = nn.Sequential(
            
   
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, kernel_size=5, stride=1),
            nn.Sigmoid()
        )


        
    
    #def alignmentscore(self,x,y):
    #    scores = pairwise2.align.localds(x,y,align_matrix,open=open_penalty,extend=gap_penalty)
    #    score_align = scores[0].score
    #    return score_align
        
    def latent(self,x,y):
        l1 = self.encoder(x)# 32
        l1 = l1.view(-1,64*10*10) # 32
        l1 = self.fc1(l1) # latent

        l2 = self.encoder(y)# 32
        l2 = l2.view(-1,64*10*10) # 32
        l2 = self.fc1(l2) 
        # use cos-simialrity
        #latent_dist =np.sqrt( np.sum(np.square(l1 - l2)) )
        
        
        return l1,l2

    def forward(self, x,y):
      # for dataset 1
        h1= self.encoder(x)
        h1 = h1.view(-1,64*10*10)
        z1 = self.fc1(h1)
        z1 = self.fc3(z1) # 64*10*10
        z1 = z1.view(-1,64,10,10)
        z1 = self.decoder(z1)
      # for dat2   
        h2= self.encoder(y)
        h2 = h2.view(-1,64*10*10)
        z2 =self.fc1(h2)
        z2 = self.fc3(z2) # to h dim 
        z2 = z2.view(-1,64,10,10)
        z2 = self.decoder(z2)
        
        # aligment 
        #s = self.alignmentscore(x,y)
        # latent distance
        l1,l2 = self.latent(x,y)
        
        
        
        return z1,z2,l1,l2
vaemodel=torch.load('/content/drive/MyDrive/DL/FeatureModel/Newcnn_sigmoid_5w_echo_cat_train.apx', map_location="cpu")

In [ ]:
AAs= ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
index_code = {}
code_index = {}
l_max = 20
for i in range(len(AAs)):
    index_code[i] = AAs[i]
    code_index[ AAs[i] ] = i

def oneHotEncode(seq, l_max=l_max, index_code=index_code, code_index=code_index):
    n_amino = 20
    matrix = np.zeros((l_max,n_amino)).astype(int)
    for i in range(len(seq)):
        matrix[ i , code_index[seq[i]] ] = 1
    return matrix

# pca encoded 
pca_index = pd.read_csv("/content/drive/My Drive/DL/VAE/AA_indexPCA.csv")
d=pca_index.set_index('Unnamed: 0').T.to_dict('list')


# pca normalization
data = d.items()
list_dat = list(d.values())
arr = np.array(list_dat)
ex = np.array(arr)
ex_norm = (ex-ex.min(axis=0))/(ex.max(axis=0)-ex.min(axis=0))

AAs=np.array(list(d.keys()))
new_pca = {}

for i in np.arange(20):
    new_pca[AAs[i]]=ex_norm[i]

new_pca
d= new_pca

def AAindexEncoding(Seq):
    length_seq=len(Seq)
    global l_max
    AAE=np.zeros([l_max,20])
    if length_seq<l_max:
        for amino in range(length_seq):
            AA=Seq[amino]# 
            AAE[amino,]=d[AA] # add PC value 
            
        for amino in range(length_seq,l_max):
            AAE[amino,]=np.zeros(20)
    else: 
        for amino in range(length_seq): # zero padding
            AA=Seq[amino]# 
            AAE[amino,]=d[AA]
        
    #AAE=np.transpose(AAE.astype(np.float32)) # row as PC. and column as AA sequence 
    return AAE 

  
def GetFeatures(file):
    hot_encode=[]
    for seq in file:
        hot_encode.append(AAindexEncoding(seq))
    hot_encode=np.array(hot_encode,dtype=np.float32)
    result=np.array(hot_encode,dtype=np.float32)
    return(result)

In [ ]:
data_files = r"/content/drive/My Drive/DL/Lung_data/"
df_mango= pd.DataFrame()
all_files = glob.glob(os.path.join(data_files , "*Brain_Met.txt"))
brain =[]
counts1=[]
for filename in all_files:
    df = pd.read_csv(filename,delimiter='\t',header=None,names=['seq','count'])
    df=df.iloc[1:,:]
    df['length'] = [len(seq) for seq in df['seq']]
    df= df[ df['length']<=20 ]
    count=len(df['seq']) # number of seqs 
    brain.append(df)
    counts1.append(count)
brain_data= pd.concat(brain, axis=0, ignore_index=True)

In [ ]:
lung_file =glob.glob(os.path.join(data_files , "*Lung.txt"))
lung=[]
counts2=[]
for filename in lung_file:
    df = pd.read_csv(filename,delimiter='\t',header=None,names=['seq','count'])
    df=df.iloc[1:,:]
    df['length'] = [len(seq) for seq in df['seq']]
    df= df[ df['length']<=20 ]
    count=len(df['seq']) 
    lung.append(df)
    counts2.append(count)
lung_data= pd.concat(lung, axis=0, ignore_index=True)

In [ ]:
seq_brain=list(brain_data['seq'])
seq_lung=list(lung_data['seq'])
AA_b1=GetFeatures(seq_brain)
AA_b2=GetFeatures(seq_brain)
AA_l1=GetFeatures(seq_lung)
AA_l2=GetFeatures(seq_lung)

train_loaderx= DataLoader(torch.from_numpy(AA_b1).float(), batch_size=1000, shuffle=False)
train_loadery = DataLoader(torch.from_numpy(AA_b2).float(),batch_size=1000,shuffle=False)

lung_loaderx= DataLoader(torch.from_numpy(AA_l1).float(), batch_size=1000, shuffle=False)
lung_loadery = DataLoader(torch.from_numpy(AA_l2).float(),batch_size=1000,shuffle=False)

In [ ]:
latent1=[]
latent2=[]
with torch.no_grad():
    for batch_idx, (x, y) in enumerate(zip(train_loaderx, train_loadery)):
        x=x.view(len(x),1,20,20).to(device)
        y=y.view(len(y),1,20,20).to(device)
        x_hat,y_hat,l1,l2= vaemodel(x,y)
        #dis = torch.nn.functional.pairwise_distance(l1, l2,2)

        latent1.append(l1)
        latent2.append(l2)




print('=========calculate distance=================')
stacked_l1 = torch.cat(latent1)
stack_l1 =stacked_l1.cpu().detach().numpy()

In [ ]:
latent3=[]
latent4=[]
with torch.no_grad():
    for batch_idx, (x, y) in enumerate(zip(lung_loaderx, lung_loadery)):
        x=x.view(len(x),1,20,20).to(device)
        y=y.view(len(y),1,20,20).to(device)
        x_hat,y_hat,l1,l2= vaemodel(x,y)
        #dis = torch.nn.functional.pairwise_distance(l1, l2,2)
        latent3.append(l1)
        latent4.append(l2)
stacked_l3 = torch.cat(latent3)

stack_l3 =stacked_l3.cpu().detach().numpy()

In [ ]:
sequences= np.concatenate((stack_l1,stack_l3),axis=0)

In [ ]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))
normal_seq=NormalizeData(sequences)
#normal_seq = torch.from_numpy( np.array(normal_seq)).float()
seqnumber=counts1+counts2

In [2]:
from itertools import chain, repeat

In [3]:
A= np.arange(1,19)

In [58]:
samples=list(chain.from_iterable((repeat(item, cnt) for item, cnt in zip(A,seqnumber))))
label=[0,1]
labels=list(itertools.chain.from_iterable(itertools.repeat(x, 9) for x in label))
sequence_dat=pd.DataFrame(normal_seq, columns = [str(1+n) for n in range(32)])
sequence_dat.insert(loc=0, column='bags', value=samples)
sequence_dat.to_csv('/content/drive/My Drive/DL/latentdata.csv')

NameError: ignored

In [11]:
def load_data(path,balance_sample_num):
  dat = pd.read_csv(path+"/latentdata.csv",sep=',')
  index, cnt =np.unique(dat['bags'],return_counts=True)
  data_list=[]
  for i in index:
    data_list.append(dat.loc[dat['bags']==1,'1':].to_numpy().tolist())
  seq_array = np.array(data_list)
  label=[0,1]
  labels=list(itertools.chain.from_iterable(itertools.repeat(x, balance_sample_num) for x in label))
  label_array=np.array(labels)

  data = torch.tensor(seq_array, dtype=torch.float32)
  target = torch.tensor(label_array,dtype=torch.float32)
  dataset_all = TensorDataset(data,target)


  return data_list,dataset_all

In [12]:
data_list, dataset_all=load_data(r'/content/drive/My Drive/DL',9)

In [54]:
train_loader = DataLoader(dataset_all, batch_size= 5)

In [14]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.L =32
        self.K = 1
        self.fclayers=nn.Sequential(
            nn.Linear(self.L, self.L),
            nn.Dropout(0.5),
            nn.ReLU(), # nn,SELU()
            nn.Linear(self.L, self.L),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(self.L, self.L),
            nn.Dropout(0.5),
            nn.ReLU()
            
        )
        self.attention = nn.Linear(self.L,self.K)
        self.sparseatt = Sparsemax(dim=1)

        self.out = nn.Sequential(
            nn.Linear(self.L*self.K,self.L) ,
            nn.BatchNorm1d(self.L),
            nn.ReLU())
        
        self.classifier = nn.Sequential(
            nn.Linear(self.L * self.K, 1), 
            nn.Sigmoid()
        )
    
    def forward(self,x):
        out = self.fclayers(x) #32

        attention = torch.squeeze(self.attention(out)) # 1 
        attention = torch.transpose(attention,0,1)

     # attention =self.sparseatt(attention)
        attention =torch.softmax(attention,dim=1)
      #attention_bag= torch.softmax(attention,dim=1) 
      
        out = torch.transpose(out,0,1)
        new_features = torch.bmm(torch.unsqueeze(attention, 1), out)
        new_features = torch.squeeze(new_features)
      
        sample_feature =self.out(new_features)
        predictions = self.classifier(sample_feature) 
        Y_hat = torch.ge(predictions, 0.5).float() 
        return predictions, attention


     # attention = torch.transpose(attention,0,1)


      # * original to get new futures 





    

In [15]:
model = Attention()

In [43]:
def loss_function( label, prediction ):
    reproduction_loss = nn.functional.binary_cross_entropy(label, prediction, reduction='sum')
    return reproduction_loss
       
def calculate_classification_error(Y_hat,Y):
    Y = Y.float()
    error = 1. - Y_hat.eq(Y).cpu().float().mean().data

    return error, Y_hat  

In [ ]:
len(data)

In [36]:
#start train 
from torch.optim import Adam
optimizer = Adam(model.parameters(), lr=1e-3)  
from sklearn.metrics import accuracy_score
epochs=1000

In [ ]:
der

In [49]:
for data, target in enumerate(train_loader):
  

AttributeError: ignored

In [53]:
for batch_idx, (data,target) in enumerate(train_loader):

SyntaxError: ignored

In [55]:
for batch_idx, (data, target) in enumerate(train_loader):
  data, target = data.to(device=device, dtype=torch.float32), target.to(device=device)

In [57]:
train_loss= []
for epoch in range(epochs):
    overall_loss =0
    train_error = 0 
    for batch_idx, (data, target) in enumerate(train_loader):
      data,target = torch.transpose(data,1,0).to(device),target.to(device) #[5]
      optimizer.zero_grad()
     
      predicitons, attention = model(data)
      predicitons =torch.squeeze(predicitons)#[5]
      loss =loss_function(predicitons,target)
      overall_loss += loss.item()
      #print(overall_loss)
      loss.backward()
      optimizer.step()
    train_loss.append(overall_loss/len(dataset_all))

    if (epoch % 100 == 0):
      print("\tEpoch", epoch , "complete!", "\tAverage Loss: ", train_loss[epoch])
    

   #[5,1]
    #Y_hat.shape 10*1
    #print(label_true[:5])
    #print(Y_hat[:5])
    #loss = loss_function(predicitons, target)
    #overall_loss +=loss.item()
    
    #error, predicted_label = calculate_classification_error(Y_hat, label_true)
    #train_error += error
    
    #loss.backward()
  
    #()if (epoch % 100 == 0):
      #print('Train Set, Epoch: {}, Loss: {:.4f},Error: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, overall_loss,train_error,accuracy_score(label_true, Y_hat)*100))    
   



	Epoch 0 complete! 	Average Loss:  0.6942630343967013
	Epoch 100 complete! 	Average Loss:  0.6947920984692044
	Epoch 200 complete! 	Average Loss:  0.7006981505288018
	Epoch 300 complete! 	Average Loss:  0.6927659511566162
	Epoch 400 complete! 	Average Loss:  0.6935693820317587


KeyboardInterrupt: ignored